En esta libreta se agrega información asociada a cada sección electoral en Sonora, ya sea a partir de las elecciones del 2018 o de datos más recientes.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

Primero vamos a extraer información del [*Estudio Censal de Participación Ciudadana* para las elecciones federales del 2018](https://www.ine.mx/transparencia/datos-abiertos/#/archivo/participacion-ciudadana-2009-2018).

Los datos se encuentran en el directorio ECPC y consiste de dos archivos separados por coma:
- `CC2018_ES_SEXO_EDAD_PARTE1.csv`
- `CC2018_ES_SEXO_EDAD_PARTE2.csv`

Este proceso puede ser algo tardado, por lo que una vez normalizados los datos se generará una versión serializada de la tabla de datos. Para regenerar esta tabla, elimine el comentario en la última linea del siguiente bloque de código y ejecútalo.

In [2]:
def parse_ecpc_date(x):
    return datetime.strptime(x, "%d/%m/%Y")

def read_ecpc(file):
    return pd.read_csv(file, sep=";", na_values=[" "],
                       parse_dates=["FELECCION"],
                       date_parser=parse_ecpc_date,
                       dtype={
                           "AELEC": "Int64",
                           "EDOCVE": "Int64",
                           "MPIOCVE": "Int64",
                           "SECCION": "Int64",
                           "TIPOSEC": "category",
                           "DEF": "Int64",
                           "DEL": "Int64",
                           "SEXO": "category",
                           "EDAD": "Int64",
                           "LN": "Int64",
                           "SV": "Int64",
                           "NV": "Int64",
                           "NS": "Int64"
                       })

def rename_ecpc(df):
    return pd.DataFrame({
        "año de elección": df["AELEC"],
        "fecha de la jornada": df["FELECCION"],
        "clave entidad federativa": df["EDOCVE"],
        "entidad federativa": df["EDONOM"],
        "clave municipio": df["MPIOCVE"],
        "municipio": df["MPIONOM"],
        "sección": df["SECCION"],
        "tipo de sección": pd.Categorical(df["TIPOSEC"]).rename_categories({
            "M": "Mixta",
            "U": "Urbana",
            "R": "Rural"
        }),
        "distrito electoral federal": df["DEF"],
        "distrito electoral local": df["DEL"],
        "sexo": df["SEXO"],
        "edad": df["EDAD"],
        "lista nominal": df["LN"],
        "si voto": df["SV"],
        "no voto": df["NV"],
        "registros no especificados": df["NS"]
    })

def serialize_ecpc():
    pt1 = read_ecpc("ECPC/CC2018_ES_SEXO_EDAD_PARTE1.csv")
    pt2 = read_ecpc("ECPC/CC2018_ES_SEXO_EDAD_PARTE2.csv")
    
    df = pd.concat([pt1, pt2], ignore_index=True)
    
    rename_ecpc(df).to_pickle("ECPC.pkl")

# serialize_ecpc()

Se carga la tabla de datos serializada del estudio censal de participación ciudadana del 2018

In [3]:
ecpc_df = pd.read_pickle("ECPC.pkl")
ecpc_df

,año de elección,fecha de la jornada,clave entidad federativa,entidad federativa,clave municipio,municipio,sección,tipo de sección,distrito electoral federal,distrito electoral local,sexo,edad,lista nominal,si voto,no voto,registros no especificados
0,2018,2018-07-01,1,AGUASCALIENTES,1,Aguascalientes,1,Urbana,3,6,0,18,24,18,6,0
1,2018,2018-07-01,1,AGUASCALIENTES,1,Aguascalientes,1,Urbana,3,6,0,19,23,20,3,0
2,2018,2018-07-01,1,AGUASCALIENTES,1,Aguascalientes,1,Urbana,3,6,0,20,23,19,4,0
3,2018,2018-07-01,1,AGUASCALIENTES,1,Aguascalientes,1,Urbana,3,6,0,21,19,12,7,0
4,2018,2018-07-01,1,AGUASCALIENTES,1,Aguascalientes,1,Urbana,3,6,0,22,19,10,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9251117,2018,2018-07-01,32,ZACATECAS,17,Guadalupe,1909,Urbana,4,3,1,65,1,1,0,0
9251118,2018,2018-07-01,32,ZACATECAS,17,Guadalupe,1909,Urbana,4,3,1,66,1,1,0,0
9251119,2018,2018-07-01,32,ZACATECAS,17,Guadalupe,1909,Urbana,4,3,1,72,1,0,1,0
9251120,2018,2018-07-01,32,ZACATECAS,17,Guadalupe,1909,Urbana,4,3,1,77,1,0,1,0


Ahora procedemos a procesar la información del [Padrón Electoral y Lista Nominal para las secciones en Sonora por distritos locales](https://www.ieesonora.org.mx/elecciones/estadistica_y_cartografia_electoral/division_distrital).

En Sonora hay 21 distritos locales y la información se encuentra disponible en formato para Excel:

- [Distrito 1](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito1.xls)
- [Distrito 2](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito2.xls)
- [Distrito 3](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito3.xls)
- [Distrito 4](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito4.xls)
- [Distrito 5](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito5.xls)
- [Distrito 6](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito6.xls)
- [Distrito 7](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito7.xls)
- [Distrito 8](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito8.xls)
- [Distrito 9](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito9.xls)
- [Distrito 10](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito10.xls)
- [Distrito 11](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito11.xls)
- [Distrito 12](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito12.xls)
- [Distrito 13](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito13.xls)
- [Distrito 14](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito14.xls)
- [Distrito 15](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito15.xls)
- [Distrito 16](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito16.xls)
- [Distrito 17](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito17.xls)
- [Distrito 18](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito18.xls)
- [Distrito 19](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito19.xls)
- [Distrito 20](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito20.xls)
- [Distrito 21](https://www.ieesonora.org.mx/documentos/estadistica_cartografia/distrito/distrito21.xls)

Estos archivos se encuentran en el directorio `PELN`

In [4]:
def read_peln(file):
    return pd.read_excel(file, skiprows=6, skipfooter=1)

def serialize_peln():
    dfs = [
        read_peln("PELN/distrito" + str(i) + ".xls")
        for i in range(1, 22)
    ]
    df = pd.concat(dfs, ignore_index=True)
    df.to_pickle("PELN.pkl")

# serialize_peln()

In [5]:
peln_df = pd.read_pickle("PELN.pkl")
peln_df

,Municipio,Sección,Lista Nominal,Lista Nominal Hombres,Lista Nominal Mujeres,Padrón Electoral,Padrón Electoral Hombres,Padrón Electoral Mujeres
0,SAN LUIS RIO COLORADO,652,802,447,355,809,450,359
1,SAN LUIS RIO COLORADO,653,176,97,79,176,97,79
2,SAN LUIS RIO COLORADO,654,279,162,117,281,163,118
3,SAN LUIS RIO COLORADO,655,481,229,252,489,231,258
4,SAN LUIS RIO COLORADO,656,747,353,394,749,355,394
...,...,...,...,...,...,...,...,...
1528,YECORA,1318,254,142,112,256,143,113
1529,YECORA,1319,708,361,347,719,370,349
1530,YECORA,1320,463,242,221,469,246,223
1531,YECORA,1321,266,140,126,270,143,127


Nos interesa filtrar los datos de la tabla ECPC que corresponden a Sonora y de ellos solo considerar la sección, el tipo de sección, su distrito electoral federal, el distrito local y la clave del municipio al que corresponden.

In [6]:
ecpcson_df = ecpc_df[ecpc_df["entidad federativa"] == "SONORA"]
ecpcson_df = pd.DataFrame({
    "Sección": ecpcson_df["sección"],
    "Tipo": ecpcson_df["tipo de sección"],
    "DEF": ecpcson_df["distrito electoral federal"],
    "DEL": ecpcson_df["distrito electoral local"],
    "Municipio ID": ecpcson_df["clave municipio"],
    "Municipio": ecpcson_df["municipio"],
})

ecpcson_df.drop_duplicates(inplace=True, ignore_index=True)

Los datos de la tabla PELN ya corresponden únicamente a Sonora, solo debemos considerar las columnas que nos son de interés, en este caso la sección y la lista nominal.

In [7]:
pelnr_df = pd.DataFrame({
    "Sección": peln_df["Sección"],
    "Lista Nominal": peln_df["Lista Nominal"]
})

Ahora, concatenamos estas dos tablas de datos considerando el número de sección de cada entrada como el índice. El resultado es la tabla de atributos que asociaremos a la cartografía seccional de Sonora.

In [8]:
atributos = pd.concat([
    ecpcson_df.set_index("Sección"),
    pelnr_df.set_index("Sección")
], axis=1, join="inner")

atributos

,Tipo,DEF,DEL,Municipio ID,Municipio,Lista Nominal
Sección,,,,,,
1,Mixta,4,18,1,Aconchi,1570
2,Rural,4,18,1,Aconchi,790
3,Urbana,2,7,2,Agua Prieta,1257
4,Urbana,2,7,2,Agua Prieta,345
5,Urbana,2,7,2,Agua Prieta,613
...,...,...,...,...,...,...
1552,Urbana,5,11,49,Hermosillo,1831
1553,Urbana,5,12,49,Hermosillo,1238
1554,Urbana,5,11,49,Hermosillo,1530


Finalmente, guardamos esta tabla de datos como valores separados por coma para importarlo en QGIS.

In [9]:
atributos.to_csv("Atributos.csv")

La cartografía electoral de Sonora se basa en la [Cartografía de Distritación Federal del INE 2016-2017](https://cartografia.ife.org.mx/sige7/?distritacion=federal) en particular en el archivo [EDMS de Sonora](https://cartografia.ife.org.mx//descargas/distritacion2017/federal/26/26.zip).

El directorio `EDMS` contiene los archivos descomprimidos, en particular nos es de interés únicamente los archivos con nombre `SECCION`.

Para incorporar estos archivos cartográficos con la tabla de atributos se realiza lo siguiente, primero se copian los archivos `SECCION` con extensión `dbf`, `prj`, `qpj`, `shp` y `shx` a la raíz de este repositorio con el nombre `Cartografia`.

Abre el programa QGIS, crea un nuevo proyecto y guardalo con el nombre `Cartografia.qgz` en la raíz de este repositorio.

Luego agrega una nueva capa vectorial.

![Agregar una nueva capa vectorial](imgs/01.png)

Elegimos el archivo `Cartografia.shp` como el archivo fuente de los datos y hacermos clic en *Agregar*.

![Cargar archivo SHP](imgs/02.png)

Cerramos la ventana y activamos la edición de esta nueva capa haciendo clic derecho en ella y seleccionando *Toggle Editing*.

![Activar edición](imgs/03.png)

Luego abrimos la ventana de propiedades de la capa, haciendo clic derecho en ella

![Abrir ventana de propiedades](imgs/04.png)

En la pestaña de *Campos* seleccionar todos excepto `seccion` y eliminarlos haciendo clic al icono mostrado en la siguiente imagen.

![Eliminar campos innecesarios](imgs/05.png)

Hacer doble clic en el campo de nombre de la sección para nombrarlo `Sección`.

![Renombrar campo a Sección](imgs/06.png)

Haz clic en *Aplicar* y luego en *Ok* para cerrar la ventana.

Agrega una nueva capa de texto delimitado.

![Agregar nueva capa de texto delimitado](imgs/07.png)

Elige el archivo `Atributos.csv` generado previamente y asegurate de elegir *Sin geometría* en la sección de definición de geometría, después haz clic en *Agregar*.

![Cargar archivo de atributos](imgs/08.png)

Abre la ventana de propiedades de la capa `Cartografia`, dirigete a la pestaña de *Joins* y haz clic en el ícono con el símbolo +.

![Abrir Joins](imgs/09.png)

Elige la capa de atributos y la columna de `Sección` como campo de agregación y objetivo. Asegurate de elegir un prefijo personalizado que esté vacío y haz clic en *Ok* para hacer el join y luego en *Aplicar* y *Ok* para cerrar la ventana de propiedades.

![Hacer Join](imgs/10.png)

Guarda los cambios realizados en las capas haciendo clic derecho en la capa de `Cartografia`.

![Guardar cambios](imgs/11.png)

Desactiva la edición de la capa `Cartografia`.

![Desactivar edicion](imgs/12.png)

Finalmente, guarda el proyecto y cierra QGIS.

![Guardar](imgs/13.png)